Problem statement
- given two JSON files **Frame A** *(list of 3D points at time $t$)* and **Frame B** *(list of 3D points at time $t+\Delta t$)*, compute the translation vector $\vec{d}=(dx,dy,dz)$, such that $Point_B = Point_A+\vec{d}$.
- such files may represent 3D coordinates of two consecutive satellite frames

Solution
- since there is no rotation, all points shift by the same vector. If we had one point $\rightarrow$ problem is trivial
- with many points, measurements have noise $\rightarrow$ solution is **least squares**

Derivation

```python
A = [A1, A2, ..., An]  # points in frame A
B = [B1, B2, ..., Bn]  # points in frame B
```

For each pair: $B_i = A_i + d_i$ $\rightarrow$ $d_i = B_i - A_i$

Because of noise, we compute the average:

- $dx = \frac{1}{N} \sum (B_{ix} - A_{ix})$
- $dy = \frac{1}{N} \sum (B_{iy} - A_{iy})$
- $dz = \frac{1}{N} \sum (B_{iz} - A_{iz})$

JSON assumption

- We assume data points are stored as follows:

```json
[
  {"x": 12.351, "y": -3.221, "z": 40.112},
  {"x": 8.154,  "y": 1.882,  "z": 37.500},
  {"x": 15.772, "y": -0.552, "z": 42.998},
  ...
]
```


Algorithm:
1. Read JSON files (*Frame A, Frame B*)
2. Extract point lists
3. Check dimensionality *(must have same size)*
4. Compute per-point differences `(diffs = B - A)`
5. Average the differences `translation = np.mean(diffs, axis=0)`
6. Output **dx, dy, dx**

In [2]:
import json
import numpy as np

def load_points(filename):
    with open(filename, "r") as f:
        data = json.load(f)

    if isinstance(data, list):
        return np.array([[p["x"], p["y"], p["z"]] for p in data])
    else:
        raise ValueError("Unrecognized JSON structure.")


In [3]:
# debug
load_points("frameA.json")

array([[12.351, -3.221, 40.112],
       [ 8.154,  1.882, 37.5  ],
       [15.772, -0.552, 42.998],
       [11.004,  3.331, 39.12 ],
       [ 9.887, -2.118, 41.443],
       [14.22 ,  2.005, 38.556],
       [10.665, -1.777, 43.331],
       [ 7.442,  0.992, 36.804]])

In [4]:
def compute_translation(A, B):
    if A.shape != B.shape:
        raise ValueError("Points lists must have same length")

    diffs = B - A
    return np.mean(diffs, axis = 0)


Test

In [5]:
A = load_points("frameA.json")
B = load_points("frameB.json")

dx, dy, dz = compute_translation(A, B)

print("Translation vector:")
print(f"dx = {dx:.4f}, dy = {dy:.4f}, dz = {dz:.4f}")

Translation vector:
dx = 2.4999, dy = -1.0015, dz = 4.2034
